# TP2: Programación Dinámica

#### Funciones auxiliares

In [71]:
# Recibe el path con el archivo a leer.
# Leemos y devolvemos dos arrays. 
# - El primero correspondiente a los e_i (ganancias de cada dia)
# - El segundo a los s_i (energía que se cuenta al día i de estar entrenando sin haber descansando previamente)
def obtener_ganancias_y_energias(path):
    with open(path, "r") as archivo:
        lineas = archivo.readlines()

    ganancias_e_i = []
    energias_s_i = []
    i = 0
    cantidad_elementos = 0

    for linea in lineas:
      if i == 0:
        cantidad_elementos = int(linea)
      else:
        if i <= cantidad_elementos:
          ganancias_e_i.append(int(linea))
        else:
          energias_s_i.append(int(linea))
      i += 1
      
    return ganancias_e_i, energias_s_i

In [72]:
#Paths de los archivos a utilizar
path_3 = 'sets/catedra/3.txt'
path_10 = 'sets/catedra/10.txt'
path_10_bis = 'sets/catedra/10_bis.txt'
path_10_todo_entreno = 'sets/catedra/10_todo_entreno.txt'
path_50 = 'sets/catedra/50.txt'
path_50_bis = 'sets/catedra/50_bis.txt'
path_100 = 'sets/catedra/100.txt'
path_500 = 'sets/catedra/500.txt'
path_1000 = 'sets/catedra/1000.txt'
path_5000 = 'sets/catedra/5000.txt'

In [73]:
# Devuelve un array con los resultados esperados de los archivos brindados por la catedra
def obtener_resultados_esperados():
    path = 'sets/catedra/Resultados Esperados.txt'
    with open(path, 'r') as archivo:
        contenido = archivo.read() 

    # Separa el contenido en bloques
    bloques = contenido.split('\n\n')
    resultados_esperados = []

    # Itera a través de cada bloque y busca "Ganancia maxima" y "Plan de entrenamiento" dentro de cada uno
    for bloque in bloques:
        nombre_archivo = ''
        ganancia_maxima = 0
        plan_entrenamiento = []

        lineas = bloque.split('\n')
        for linea in lineas:
            if ".txt" in linea:
                nombre_archivo = linea
            if linea.startswith('Ganancia maxima:'):
                ganancia_maxima = int(linea.split(':')[1].strip())
            if linea.startswith('Plan de entrenamiento:'):
                plan_entrenamiento = [accion.strip() for accion in linea.split(':')[1].strip().split(',')]
        
        resultados_esperados.append({
            'nombre_archivo': nombre_archivo,
            'ganancia_maxima': ganancia_maxima,
            'plan_entrenamiento': plan_entrenamiento
        })

    return resultados_esperados


#### Funciones de resolución 

In [85]:
# Recibe las ganancias de cada día (e_i), las energias con las que se cuenta cada día (s_i), el dia actual y la cantidad de días entrenados.
# Devuelve la ganancia obtenida en el dia según la energía disponible.
def ganancia_dia_actual(e_i, s_i, dia, dias_entrenados):
  return min(e_i[dia], s_i[dias_entrenados])

def reconstruir_solucion(ganancias, ganancia_maxima):
  n = len(ganancias) - 1
  solucion = []
  # recorrer la matriz bajando siempre por la diagonal hasta llegar a la fila 0,
  # luego ir al maximo de la anterior fila
  dia = n - 1 # columnas
  dias_entrenados = ganancias[n].index(ganancia_maxima) - 1 # filas
  solucion.append('Entreno')
  while dia >= 0:
    while dias_entrenados >= 0 and dia >= 0:
      solucion.append('Entreno')
      dias_entrenados -= 1
      dia -= 1
    if dia >= 0:
      solucion.append('Descanso')
      # ir al maximo de la columna anterior
      dia -= 1
      dias_entrenados = ganancias[dia].index(max(ganancias[dia]))
  return solucion[::-1] # invertir la solucion

# Recibe las ganancias de cada día (e_i) y las energias con las que se cuenta cada día (s_i).
# Devuelve la ganancia máxima que se puede obtener a lo largo de varios días tomando decisiones 
# óptimas sobre cómo asignar la energía disponible
def ganancia_maxima_posible(e_i, s_i):

  cantidad_dias = len(e_i) # n

  ganancias_obtenidas = [[0] * cantidad_dias for _ in range(cantidad_dias)] # matriz de n*n
  dias_entrenados = 2
  ganancias_obtenidas[0][0] = ganancia_dia_actual(e_i, s_i, 0, 0)
  # inicializamos tmb el día 2 porque necesita el resultado de n = 0 (y se va de la matriz)
  ganancias_obtenidas[1][0] = ganancia_dia_actual(e_i, s_i, 1, 0)
  ganancias_obtenidas[1][1] = ganancia_dia_actual(e_i, s_i, 1, 1) + ganancias_obtenidas[0][0]

  for dia in range(2, cantidad_dias):
    ganancias_obtenidas[dia][0] = ganancia_dia_actual(e_i, s_i, dia, 0)
    ganancias_obtenidas[dia][0] += max(ganancias_obtenidas[dia-2])
    for dia_entrenado in range(1, dias_entrenados+1):
      ganancias_obtenidas[dia][dia_entrenado] = ganancia_dia_actual(e_i, s_i, dia, dia_entrenado) + ganancias_obtenidas[dia-1][dia_entrenado-1]
    dias_entrenados += 1
  return ganancias_obtenidas, max(ganancias_obtenidas[cantidad_dias-1])


### Pruebas

#### Sets de la catedra 

In [75]:
e_i_3, s_i_3 = obtener_ganancias_y_energias(path_3)
e_i_10, s_i_10 = obtener_ganancias_y_energias(path_10)
e_i_10_bis, s_i_10_bis = obtener_ganancias_y_energias(path_10_bis)
e_i_10_todo_entreno, s_i_10_todo_entreno = obtener_ganancias_y_energias(path_10_todo_entreno)
e_i_50, s_i_50 = obtener_ganancias_y_energias(path_50)
e_i_50_bis, s_i_50_bis = obtener_ganancias_y_energias(path_50_bis)
e_i_100, s_i_100 = obtener_ganancias_y_energias(path_100)
e_i_500, s_i_500 = obtener_ganancias_y_energias(path_500)
e_i_1000, s_i_1000 = obtener_ganancias_y_energias(path_1000)
e_i_5000, s_i_5000 = obtener_ganancias_y_energias(path_5000)

In [76]:
g3, ganancia_max_3_dias = ganancia_maxima_posible(e_i_3, s_i_3)
g10, ganancia_max_10_dias = ganancia_maxima_posible(e_i_10, s_i_10)
g10bis, ganancia_max_10_dias_bis = ganancia_maxima_posible(e_i_10_bis, s_i_10_bis)
g10te, ganancia_max_10_dias_todo_entreno = ganancia_maxima_posible(e_i_10_todo_entreno, s_i_10_todo_entreno)
g50, ganancia_max_50_dias = ganancia_maxima_posible(e_i_50, s_i_50)
g50bis, ganancia_max_50_dias_bis = ganancia_maxima_posible(e_i_50_bis, s_i_50_bis)
g100, ganancia_max_100_dias = ganancia_maxima_posible(e_i_100, s_i_100)
g500, ganancia_max_500_dias = ganancia_maxima_posible(e_i_500, s_i_500)
g1000, ganancia_max_1000_dias = ganancia_maxima_posible(e_i_1000, s_i_1000)
g5000, ganancia_max_5000_dias = ganancia_maxima_posible(e_i_5000, s_i_5000)

In [88]:
planes_de_entrenamiento_optimos_obtenidos = [
reconstruir_solucion(g3, ganancia_max_3_dias),
reconstruir_solucion(g10, ganancia_max_10_dias),
reconstruir_solucion(g10bis, ganancia_max_10_dias_bis),
reconstruir_solucion(g10te, ganancia_max_10_dias_todo_entreno),
reconstruir_solucion(g50, ganancia_max_50_dias),
reconstruir_solucion(g50bis, ganancia_max_50_dias_bis),
reconstruir_solucion(g100, ganancia_max_100_dias),
reconstruir_solucion(g500, ganancia_max_500_dias),
reconstruir_solucion(g1000, ganancia_max_1000_dias),
reconstruir_solucion(g5000, ganancia_max_5000_dias)]


In [78]:
resultados_esperados = obtener_resultados_esperados()

ganancias_obtenidas = [ganancia_max_3_dias, ganancia_max_10_dias, ganancia_max_10_dias_bis, ganancia_max_10_dias_todo_entreno,\
                      ganancia_max_50_dias, ganancia_max_50_dias_bis, ganancia_max_100_dias, ganancia_max_500_dias, ganancia_max_1000_dias,\
                      ganancia_max_5000_dias]

In [89]:
# correr pip install tabulate en consola para poder usar esta libreria 
from tabulate import tabulate

# Imprime los resultados obtenidos y los esperados en una tabla
resultados = []
i = 0
for ganancia_esperada in resultados_esperados:
  es_correcto = "Sí" if ganancia_esperada['ganancia_maxima'] == ganancias_obtenidas[i] and planes_de_entrenamiento_optimos_obtenidos[i] == ganancia_esperada['plan_entrenamiento'] else "No"
  resultados.append([ganancia_esperada['nombre_archivo'], ganancia_esperada['ganancia_maxima'], ganancias_obtenidas[i], es_correcto])
  i += 1

print(tabulate(resultados, headers=['Archivo', 'Ganancia Esperada', 'Ganancia Obtenida', 'Se obtuvo el resultado esperado?']))

Archivo                Ganancia Esperada    Ganancia Obtenida  Se obtuvo el resultado esperado?
-------------------  -------------------  -------------------  ----------------------------------
3.txt                                  7                    7  Sí
10.txt                               380                  380  Sí
10_bis.txt                           523                  523  Sí
10_todo_entreno.txt                  860                  860  Sí
50.txt                              1870                 1870  Sí
50_bis.txt                          2136                 2136  Sí
100.txt                             5325                 5325  Sí
500.txt                            27158                27158  Sí
1000.txt                           54021                54021  Sí
5000.txt                          279175               279175  Sí


Reconstruímos las soluciones